In [1]:
# import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

import re, string
import emoji
import nltk

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

from transformers import BertTokenizerFast
from transformers import RobertaTokenizerFast

In [2]:
base_path = r'C:/Users/Admin/Desktop/Coronavirus Tweets'

In [3]:
df = pd.read_csv(base_path + '/Data/Corona_NLP_train.csv', encoding='ISO-8859-1')
df_test = pd.read_csv(base_path + '/Data/Corona_NLP_test.csv')
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [5]:
df = df[['OriginalTweet','Sentiment']]
df_test = df_test[['OriginalTweet','Sentiment']]

## Defind functions to clean data

In [6]:
# remove emojis
def strip_emoji(text):
    
    return re.sub(emoji.get_emoji_regexp(), r"", text)


# remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text):
    # remove \n and \r and lowercase
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() 
    
    # remove links and mentions
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) 
    
    # remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    text = re.sub(r'[^\x00-\x7f]',r'', text)
    
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    
    return text


#clean hashtags
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) 
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet))
    
    return new_tweet2


# filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
            
    return ' '.join(sent)


# remove multiple spaces
def remove_mult_spaces(text): 
    
    return re.sub("\s\s+" , " ", text)

In [7]:
#Remove stopwords & Punctuations
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))


def punct_remove(text):
    punct = re.sub(r"[^\w\s\d]","", text)
    
    return punct


def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [8]:
# function to correct misspellings and out of vocab words
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {'colour':'color',
                'centre':'center',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'cancelled':'canceled',
                'labour':'labor',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium',
                'Snapchat': 'social medium',
                'quora': 'social medium',
                'Quora': 'social medium',
                'mediumns': 'mediums',
                'bitcoin': 'currency',
                'cryptocurrency': 'currency',
                'upsc': 'union public service commission',
                'mbbs': 'bachelor medicine',
                'ece': 'educational credential evaluators',
                'aiims': 'all india institute medical science',
                'iim': 'india institute management',
                'sbi': 'state bank india',
                'blockchain': 'crytography',
                'and': '',
                'reducational':'educational',
                'neducational':'educational',
                'greeducational': 'greed educational',
                'pieducational': 'educational',
                'deducational': 'educational',
                'Quorans': 'Quoran',
                'stayhome': 'stay at home',
                'stayathome': 'stay at home',
                'toiletpaper': 'toilet paper',
                'covid_': 'covid',
                'pemic': 'pandemic',
                'panicbuying': 'panic buying',
                'quarantinelife': 'quarantine life',
                'coronacrisis': 'corona crisis',
                'underst': 'underestimate',
                'hsanitizer': 'sanitizer',
                'asda': '',
                'realdonaldtrump': 'real donald trump',
                'coronavirusuk': 'virus',
                'evaluatorsssion': 'evaluate',
                'evaluatorsdented': 'evaluate',
                'unpeducational': 'no educational',
                'pemic': 'pandemic',
                'coronavirususa': 'virus',
                'coronavirus': 'virus',
                'buyinguk': 'buy',
                'stoppanic': 'stop panic',
                'staysafe': 'stay safe',
                'gov': 'government'
                }

mispellings, mispellings_re = _get_mispell(mispell_dict)


def replace_typical_misspell(text):
    text = text.lower()
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

In [9]:
# clean train data
texts_new = []
for t in df.OriginalTweet:
    texts_new.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(t))))))

In [ ]:
# clean test data
texts_new_test = []
for t in df_test.OriginalTweet:
    texts_new_test.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(t))))))

In [ ]:
df['text_clean'] = texts_new
df_test['text_clean'] = texts_new_test

In [ ]:
df['text_clean'] = df['text_clean'].apply(lambda x : punct_remove(x))
df['text_clean'] = df['text_clean'].apply(lambda x : remove_stopwords(x))

In [ ]:
df['text_clean'] = df['text_clean'].apply(lambda x : replace_typical_misspell(x))
df_test['text_clean'] = df_test['text_clean'].apply(lambda x : replace_typical_misspell(x))

In [ ]:
df['text_clean'].head()

In [ ]:
df_test['text_clean'].head()

In [ ]:
# get the length of text in train data
text_len = []
for text in df.text_clean:
    tweet_len = len(text.split())
    text_len.append(tweet_len)

In [ ]:
df['text_len'] = text_len

In [ ]:
# get the length of text in test data
text_len_test = []
for text in df_test.text_clean:
    tweet_len = len(text.split())
    text_len_test.append(tweet_len)

In [ ]:
df_test['text_len'] = text_len_test

In [ ]:
print(f" df shape: {df.shape}")
print(f" df_test shape: {df_test.shape}")

## Remove Tweets that have length lesser than 5

In [ ]:
df = df[df['text_len'] > 4]

In [ ]:
df_test = df_test[df_test['text_len'] > 4]

In [ ]:
print(f" df shape: {df.shape}")
print(f" df_test shape: {df_test.shape}")

## BERT Tokenizer

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

## Find tweets that are not in English

In [ ]:
token_lens = []

for txt in df['text_clean'].values:
    tokens = tokenizer.encode(txt, max_length = 512, truncation = True)
    token_lens.append(len(tokens))
    
max_len = np.max(token_lens)
print(f"Max tokenized sentence length: {max_len}")

In [ ]:
token_lens = []

for i,txt in enumerate(df['text_clean'].values):
    tokens = tokenizer.encode(txt, max_length = 512, truncation = True)
    token_lens.append(len(tokens))
    if len(tokens) > 80:
        print(f"Index: {i}, Text: {txt}")  

In [ ]:
df['token_lens'] = token_lens

In [ ]:
df = df.sort_values(by='token_lens', ascending=False)
df.head()

In [ ]:
df = df.iloc[12:]
df.head()

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
token_lens_test = []

for txt in df_test['text_clean'].values:
    tokens = tokenizer.encode(txt, max_length = 512, truncation = True)
    token_lens_test.append(len(tokens))
    
max_len=np.max(token_lens_test)
print(f"Max tokenized sentence length: {max_len}")

In [ ]:
token_lens_test = []

for i,txt in enumerate(df_test['text_clean'].values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens_test.append(len(tokens))
    if len(tokens)>80:
        print(f"Index: {i}, Text: {txt}")

In [ ]:
df_test['token_lens'] = token_lens_test

In [ ]:
df_test = df_test.sort_values(by='token_lens', ascending=False)
df_test.head(10) 

In [ ]:
df = df.iloc[5:]
df.head()

In [ ]:
df_test = df_test.sample(frac=1).reset_index(drop=True)
df_test.head() 

In [ ]:
df['Sentiment'].value_counts()

In [ ]:
df['Sentiment'] = df['Sentiment'].map({'Extremely Negative':0,'Negative':0,'Neutral':1,'Positive':2,'Extremely Positive':2})

In [ ]:
df_test['Sentiment'] = df_test['Sentiment'].map({'Extremely Negative':0,'Negative':0,'Neutral':1,'Positive':2,'Extremely Positive':2})

In [ ]:
df['Sentiment'].value_counts()

## Random Over Sampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler


ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df['text_clean']).reshape(-1, 1), 
                                    np.array(df['Sentiment']).reshape(-1, 1))


train_data = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), 
                        columns = ['text_clean', 'Sentiment']);

# Train - Validation - Test Split

In [ ]:
X = train_data['text_clean'].values
y = train_data['Sentiment'].values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

In [ ]:
X_test = df_test['text_clean'].values
y_test = df_test['Sentiment'].values

In [ ]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape

In [ ]:
np.save('./Clean data with 2 layers/X_train.npy', X_train)
np.save('./Clean data with 2 layers/y_train.npy', y_train)
np.save('./Clean data with 2 layers/X_valid.npy', X_valid)
np.save('./Clean data with 2 layers/y_valid.npy', y_valid)
np.save('./Clean data with 2 layers/X_test.npy', X_test)
np.save('./Clean data with 2 layers/y_test.npy', y_test)